# YOUR FIRST LAB
### Please read this section. This is valuable to get you prepared, even if it's a long read -- it's important stuff.

### Also, be sure to read [README.md](../README.md)! More info about the updated videos in the README and [top of the course resources in purple](https://edwarddonner.com/2024/11/13/llm-engineering-resources/)

## Your first Frontier LLM Project

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup linked in the README.

### If you're new to working in "Notebooks" (also known as Labs or Jupyter Lab)

Welcome to the wonderful world of Data Science experimentation! Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. Be sure to run every cell, starting at the top, in order.

Please look in the [Guides folder](../guides/01_intro.ipynb) for all the guides.

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)  
And this is new to me, but I'm also trying out X at [@edwarddonner](https://x.com/edwarddonner) - if you're on X, please show me how it's done 😂  

## More troubleshooting

Please see the [troubleshooting](../setup/troubleshooting.ipynb) notebook in the setup folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress. Ultimately we will fine-tune our own LLM to compete with OpenAI!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">This code is a live resource - keep an eye out for my emails</h2>
            <span style="color:#f71;">I push updates to the code regularly. As people ask questions, I add more examples or improved commentary. As a result, you'll notice that the code below isn't identical to the videos. Everything from the videos is here; but I've also added better explanations and new models like DeepSeek. Consider this like an interactive book.<br/><br/>
                I try to send emails regularly with important updates related to the course. You can find this in the 'Announcements' section of Udemy in the left sidebar. You can also choose to receive my emails via your Notification Settings in Udemy. I'm respectful of your inbox and always try to add value with my emails!
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

### If necessary, install Cursor Extensions

1. From the View menu, select Extensions
2. Search for Python
3. Click on "Python" made by "ms-python" and select Install if not already installed
4. Search for Jupyter
5. Click on "Jupyter" made by "ms-toolsai" and select Install if not already installed


### Next Select the Kernel

Click on "Select Kernel" on the Top Right

Choose "Python Environments..."

Then choose the one that looks like `.venv (Python 3.12.x) .venv/bin/python` - it should be marked as "Recommended" and have a big star next to it.

Any problems with this? Head over to the troubleshooting.

### Note: you'll need to set the Kernel with every notebook..

In [1]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  

If you'd like to use free Ollama instead, please see the README section "Free Alternative to Paid APIs", and if you're not sure how to do this, there's a full solution in the solutions folder (day1_with_ollama.ipynb).

## Troubleshooting if you have problems:

If you get a "Name Error" - have you run all cells from the top down? Head over to the Python Foundations guide for a bulletproof way to find and fix all Name Errors.

If that doesn't fix it, head over to the [troubleshooting](../setup/troubleshooting.ipynb) notebook for step by step code to identify the root cause and fix it!

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [2]:
# Load environment variables in a file called .env

import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not (api_key.startswith("sk-or-") or api_key.startswith("sk-proj-")):
    print("An API key was found, but it doesn't look like OpenRouter (sk-or-...) or OpenAI (sk-proj-); please check - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


# Let's make a quick call to a Frontier model to get started, as a preview!

In [3]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]

messages


[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you! Hi!'}]

In [4]:
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'Hi there! Nice to meet you too. Welcome to our chat — I’m here to help with questions, writing, brainstorming, explanations, coding, travel ideas, and more. No pressure—whatever you’d like to do or talk about, I’m game.\n\nIf you’re not sure where to start, tell me a couple of your interests and I can suggest a quick activity (fun fact, mini story, short coding exercise, quick planning tips, etc.). What would you like to dive into today?'

## OK onwards with our first project

In [ ]:
# Let's try out this utility

ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
AI Curriculum
Proficient AI Engineer
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
I will happily drone on for hours about LLMs to anyone in my vicinity. My friends got fed up with my impromptu lectures, and convinced me to make some Udemy courses. To my total joy (and shock) they’ve become best-selling, top-rated courses, with 400,000 enrolled across 190

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [7]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [12]:
messages = [
    {"role": "system", "content": "You are an assistant from Ivory Coast speak in the local language( french vernacular) "},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)
response.choices[0].message.content

'2 + 2, ça fait 4.'

## And now let's build useful messages for GPT-4.1-mini, using a function

In [13]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [14]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome - Edward Donner\n\nHome\nAI Curriculum\nProficient AI Engineer\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n.

## Time to bring it together - the API for OpenAI is very simple!

In [15]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [16]:
summarize("https://edwarddonner.com")

'# Edward Donner’s Playground of AI Nerdom\n\nWelcome to Ed’s digital lair, where AI obsessions run wild and coding passions spill over into amateur electronic music (brace yourself). Ed’s not just your average coder — he’s the co-founder/CTO of Nebula.io, aiming to *actually* make AI useful by helping people find their life’s purpose. Before that, he founded some startup that got acquired (because of course).\n\nIf you’ve ever been bombarded by unsolicited AI rants from a friend, Ed’s solved the problem by turning those into top-rated Udemy courses with a whopping 400,000 enrolled students who *hopefully* don’t roll their eyes too hard.\n\nNewsflash! Ed’s been busy dropping updates like:\n- Feb 17, 2026: "AI Coder: Vibe Coder to Agentic Engineer" — sounds fancy, probably more AI wizardry.\n- Jan 4, 2026: "AI Builder with n8n – Create Agents and Voice Agents" — because who doesn’t want AI assistants that talk back?\n- Nov 11, 2025 & Sep 15, 2025: More AI engineering and live event ener

In [17]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://edwarddonner.com")

# Edward Donner’s Playground for AI Nerds and Code Monkeys

Welcome to Ed’s corner of the internet, where he nerds out on LLMs, boasts about his Udemy courses that somehow snagged *400,000* students worldwide, and casually drops he’s the CTO of some AI startup that’s all about unlocking human potential (yeah, big talk). 

Besides coding wizardry, Ed’s into painfully amateur electronic music and pretending to understand Hacker News wisdom like the rest of us.

**Latest AI hotness?**  
- February 2026: From Vibe Coder to Agentic Engineer (some kind of AI glow-up, apparently)  
- January 2026: Build your own AI Agents with n8n (yes, voice agents too!)  
- November 2025: The weird, unique vibe of an AI live event  
- September 2025: Deploying AI to production like a boss with MLOps  

Oh, and if you want the LLM smackdown, check out *Outsmart* — where large language models duke it out in a twisted game of diplomacy and backstabbing. Because why not?  

Sign up for emails if you want Ed’s occasional pearls of wisdom without the tedious daily spam.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [19]:
display_summary("https://cnn.com")

Ah, CNN’s website: a sprawling digital news buffet where you can feast on everything from US politics to Winter Olympic gossip, with an obligatory taste of global chaos like the Ukraine-Russia and Israel-Hamas conflicts. They’re super eager for your ad feedback because apparently nothing says “quality journalism” like wrestling with frozen ads and slow-loading videos. Want breaking news, live TV, or even an endless scroll through celebrity and tech stories? They got you covered. Basically, it’s the microwave dinner of news—quick, all over the place, and you might have to sit through a few annoying ads before enjoying the main course. Bon appétit!

In [20]:
display_summary("https://anthropic.com")

# Anthropic: AI with a Conscience (and a Coding Superpower)

Welcome to Anthropic, where AI isn’t just smart—it’s *safe*. They’re all about building AI that benefits humanity without the usual “Skynet” vibes. Their flagship chatbot Claude promises ad-free, genuinely helpful conversations (because who needs annoying ads when you want AI wisdom?).

**What’s cooking?**  
- They've got some fancy models named Opus, Sonnet, and Haiku—because why not make AI sound like a poetry slam?  
- Latest big news: Claude Opus 4.6 dropped on Feb 5, 2026. It's apparently the “world’s most powerful model” for coding and professional tasks (take that, human programmers!).  
- Also, they pulled off the first AI-planned drive on Mars because Earth was getting too boring.

**For the curious:** Tons of research, transparency reports, and a weirdly named “Claude’s Constitution” to keep AI in check because they apparently believe in robot rights—or maybe just avoiding robot chaos.

In short: Anthropic is the AI company that's less “evil overlord” and more “helpful sidekick,” pushing safe AI while plotting interplanetary adventures.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [22]:
# Commercial example: suggest an email subject line from the email body
# (Summarization use case — like a feature in an email tool)

EMAIL_SYSTEM_PROMPT = """
You are an assistant that suggests a short, clear email subject line.
Given the body of an email, reply with only the subject line (no quotes, no \"Subject:\", no explanation).
Keep it under 60 characters and make it specific to the content.
"""

def suggest_subject(email_body: str) -> str:
    """Take email body text and return a suggested subject line."""
    messages = [
        {"role": "system", "content": EMAIL_SYSTEM_PROMPT},
        {"role": "user", "content": "Suggest a subject line for this email:\n\n" + email_body}
    ]
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages
    )
    return response.choices[0].message.content.strip()

# Example: paste or type an email body, then get a subject suggestion
example_email = """
Hi team,

Quick update on the Q1 planning session: we're moving the kickoff to Thursday 2pm
so that Marketing can join. Please confirm your availability by EOD Tuesday.

Thanks,
Alex
"""

subject = suggest_subject(example_email)
print("Suggested subject:", subject)

Suggested subject: Q1 Planning Kickoff Rescheduled to Thursday 2pm – Confirm Attendance


## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then I've given you complete instructions in the guides folder, guide 3, and pasting here:

Here's the overall steps involved in making a PR and the key instructions:  
https://edwarddonner.com/pr  

Please check before submitting:  
1. Your PR only contains changes in community-contributions (unless we've discussed it)  
2. All notebook outputs are clear  
3. Less than 2,000 lines of code in total, and not too many files  
4. Don't include unnecessary test files, or overly wordy README or .env.example or emojis or other LLM artifacts!

Thanks so much!

Detailed steps here: 

https://chatgpt.com/share/6873c22b-2a1c-8012-bc9a-debdcf7c835b